# Extract DIEM - FAO data


The notebook follows the template of https://github.com/Andrampa/DIEM_API/blob/main/DIEM_API_get_aggregated_survey_data.ipynb and it is used to extract Data in Emergencies data from the FAO database.

Use the `argcis` environment.

In [1]:
from pathlib import Path
from arcgis.gis import GIS
import re
import datetime

In [8]:
data_dir = Path("../data/raw/DIEM")

In [9]:
data_dir.resolve()

PosixPath('/Users/paoich/Documents/Work/Research/Repos/agile2025/data/raw/DIEM')

In [20]:
###USER DEFINED PARAMETERS

# Login parameters
USERNAME = "paoloandrich"
PASS = "7^VPE6ck#ovi"

# Survey parameters
iso3 = "LSO"  # ISO3 3-letters code of the country - leave empty for all countries. 
round = ""  # DIEM survey round number (integer - leave empty for all rounds)

# Download parameters
return_geometry_in_aggregated_data = False  # Include geometry in aggregated data
geometry_export_format = 'File Geodatabase'  # Export format: Shapefile, CSV, File Geodatabase, etc.

In [21]:
# Get the current date and time
current_datetime = datetime.datetime.now()
# Format the date and time as YYYYMMDDhhmmss
formatted_datetime = current_datetime.strftime("%Y%m%d%H%M%S")

# Initialize the where clause
where_clause = ""

# Build the where clause based on conditions
if iso3 and round:
    where_clause = f"adm0_iso3 = '{iso3}' and round = {round}"
elif iso3:
    where_clause = f"adm0_iso3 = '{iso3}'"
elif round:
    where_clause = f"round = {round}"
else:
    where_clause = "1=1"  # Select all records if no filters are set

In [22]:
# Connect to the GIS
gis = GIS('https://hqfao-hub.maps.arcgis.com', USERNAME, PASS)

# List of predefined item IDs
item_ids = [
    '1b006938d6a344aeb5a309f69f3e344b',
    'fbef5b1ef85840838166a6b4d359f9bb',
    '499917f1518141209c2a6de55a79d991',
    '71460258c059453d8eab2d7c56a7b0c5'
]

for item_id in item_ids:
    group_item = gis.content.get(item_id)
    group_title = "_".join([w.lower() for w in re.sub("[(),]", "", group_item.title).split(" ")])
    print(f"Processing the {group_item.title} dataset.")
    if return_geometry_in_aggregated_data:
        subset_feature_layer_id = group_item.layers[0].properties.id
        parameters = {"layers": [{"id": subset_feature_layer_id, "where": where_clause}]}
        # Export the data with geometry included
        result = group_item.export(
            f'{group_title}_{formatted_datetime}', geometry_export_format, parameters=parameters
        )
        result.download(str(data_dir.resolve()))  # Download the export to the specified directory
        result.delete()  # Clean up by deleting the export from AGOL
        print(f"Saved the {group_title} data in {str(data_dir.resolve())}.\n")
    else:
        subset_feature_layer = group_item.layers[0]
        df = subset_feature_layer.query(
            where=where_clause, out_fields="*", returnGeometry=False, as_df=True
        )
        df = df.drop(columns=['SHAPE'], errors='ignore')  # Exclude the SHAPE field if it exists
        df.to_csv(data_dir / f"{group_title}_{iso3}_{formatted_datetime}.csv")
        print(f"Saved the {group_title} data in {str(data_dir.resolve())}.\n")

print("Process completed.")

Processing the DIEM aggregated data (crop production thematic area) dataset.
Saved the diem_aggregated_data_crop_production_thematic_area data in /Users/paoich/Documents/Work/Research/Repos/agile2025/data/raw/DIEM.

Processing the DIEM aggregated data (food security thematic area) dataset.
Saved the diem_aggregated_data_food_security_thematic_area data in /Users/paoich/Documents/Work/Research/Repos/agile2025/data/raw/DIEM.

Processing the DIEM aggregated data (income, shocks and needs thematic areas) dataset.
Saved the diem_aggregated_data_income_shocks_and_needs_thematic_areas data in /Users/paoich/Documents/Work/Research/Repos/agile2025/data/raw/DIEM.

Processing the DIEM aggregated data (livestock production thematic area) dataset.
Saved the diem_aggregated_data_livestock_production_thematic_area data in /Users/paoich/Documents/Work/Research/Repos/agile2025/data/raw/DIEM.

Process completed.
